### Langchain + RAG Demo on LlaMa-2–7b + Embeddings Model using Chainlit 
##### https://medium.com/@madhur.prashant7/demo-langchain-rag-demo-on-llama-2-7b-embeddings-model-using-chainlit-559c10ce3fbf

### 0. Default Setting 

In [13]:
import chainlit as cl
import transformers
import torch
import os 
import numpy as np 
import pandas as pd 
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM

In [14]:
torch.__version__

'2.1.2'

In [15]:
default_path = os.getcwd()
data_path = os.path.join(default_path, '../data')

### 1. Env Setting

### 2. Chroma 

In [18]:
import chromadb
from chromadb.utils import embedding_functions

In [19]:
# 연결 테스트
chroma_client = chromadb.HttpClient(host='172.19.240.1', port=8000)

2023-12-26 04:56:21 - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2023-12-26 04:56:21 - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [20]:
chroma_client.list_collections()

[]

In [10]:
def custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

In [21]:
def retrieval_qa_chain(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 2}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )
    return qa_chain

In [22]:
#Loading the model
def load_llm():
    # Load the locally downloaded model here
    llm = CTransformers(
        model = "meta/Llama-2-7B-Chat-GGML",
        model_type="llama",
        max_new_tokens = 512,
        temperature = 0.5
    )
    return llm

In [24]:
#QA Model Function
def qa_bot():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cpu'})
    db = FAISS.load_local(DB_FAISS_PATH, embeddings)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, db)
    return qa

In [25]:
def final_result(query):
    qa_result = qa_bot()
    response = qa_result({'query': query})
    return response